# Quote API - Engine Documentation

Tasks to be completed:
1. Pull HTML results from stock query.
2. Parse financial information from the HTML results.
3. Export information in an organized way.
4. Package functions neatly for use in other projects.


## <a name="TOC"></a> Table of Contents:
---
1. [Proof of Concept](#proof)
2. [Fundamental Functions](#func)
3. [RESTful Functions](#REST)



In [1]:
# ------------------------- CONFIGURE ENVIRONMENT ------------------------- #

# Environment hard reset
%reset -f

# Libraries for scraping
import requests
from bs4 import BeautifulSoup
from lxml import html
from urllib.request import Request, urlopen
import urllib.request
import urllib.parse
import urllib.error
import ssl
import ast
import os

# JSON Support
import json

# Configure paths
from pathlib import Path
data_path = Path('Profiles/')


## <a name="proof"></a> [Proof of Concept](#TOC)
---

This section is built to demonstrate how the API could form queries for specific tickers and get the HTML results.


In [36]:
# ------------------------- FORM QUERY ------------------------- #

ticker = "HD"
query = "https://in.finance.yahoo.com/quote/{}?ltr=1".format(ticker)


# ------------------------- PARSE QUERY ------------------------- #

# For ignoring SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Making the website believe that you are accessing it using a Mozilla browser
req = Request(query, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()

# Creating a BeautifulSoup object of the HTML page for easy extraction of data.
soup = BeautifulSoup(webpage, 'html.parser')
html = soup.prettify('utf-8')
profile = {}
details = {}

# Present Value
for span in soup.findAll('span', attrs={'class': 'Trsdu(0.3s) Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(b)'}):
    profile['Present Value'] = span.text.strip()

# Present Growth
for div in soup.findAll('div', attrs={'class': 'D(ib) Va(t)'}):
    for span in div.findAll('span', recursive=False):
        profile['Present Growth'] = span.text.strip()

# Previous Close
for td in soup.findAll('td', attrs={'data-test': 'PREV_CLOSE-value'}):
    for span in td.findAll('span', recursive=False):
        details['Previous Close'] = span.text.strip()

# Open Value
for td in soup.findAll('td', attrs={'data-test': 'OPEN-value'}):
    for span in td.findAll('span', recursive=False):
        details['Open'] = span.text.strip()

# Bid
for td in soup.findAll('td', attrs={'data-test': 'BID-value'}):
    for span in td.findAll('span', recursive=False):
        details['Bid'] = span.text.strip()

# Ask
for td in soup.findAll('td', attrs={'data-test': 'ASK-value'}):
    for span in td.findAll('span', recursive=False):
        details['Ask'] = span.text.strip()

# Day's Range
for td in soup.findAll('td', attrs={'data-test': 'DAYS_RANGE-value'}):
    for span in td.findAll('span', recursive=False):
        details['Day Range'] = span.text.strip()

# Fifty-two Week Range
for td in soup.findAll('td', attrs={'data-test': 'FIFTY_TWO_WK_RANGE-value'}):
    for span in td.findAll('span', recursive=False):
        details['Fifty-Two Week Range'] = span.text.strip()

# Trading Volume
for td in soup.findAll('td', attrs={'data-test': 'TD_VOLUME-value'}):
    for span in td.findAll('span', recursive=False):
        details['Day Volume'] = span.text.strip()

# Average 3M Volume
for td in soup.findAll('td', attrs={'data-test': 'AVERAGE_VOLUME_3MONTH-value'}):
    for span in td.findAll('span', recursive=False):
        details['Average 3M Volume'] = span.text.strip()

# Market Capitalization
for td in soup.findAll('td', attrs={'data-test': 'MARKET_CAP-value'}):
    for span in td.findAll('span', recursive=False):
        details['Market Capitalization'] = span.text.strip()

# Beta 3Y
for td in soup.findAll('td', attrs={'data-test': 'BETA_3Y-value'}):
    for span in td.findAll('span', recursive=False):
        details['Beta 3Y'] = span.text.strip()

# PE Ratio
for td in soup.findAll('td', attrs={'data-test': 'PE_RATIO-value'}):
    for span in td.findAll('span', recursive=False):
        details['PE Ratio'] = span.text.strip()

# EPS Ratio
for td in soup.findAll('td', attrs={'data-test': 'EPS_RATIO-value'}):
    for span in td.findAll('span', recursive=False):
        details['EPS Ratio'] = span.text.strip()

# Earnings Date
for td in soup.findAll('td', attrs={'data-test': 'EARNINGS_DATE-value'}):
    details['Earnings Date'] = []
    for span in td.findAll('span', recursive=False):
        details['Earnings Date'] = span.text.strip()

# Dividend and Yield
for td in soup.findAll('td', attrs={'data-test': 'DIVIDEND_AND_YIELD-value'}):
    details['Dividend'] = td.text.strip().split()[0]
    details['Dividend Yield'] = td.text.strip().split()[1].translate({ord(i): None for i in '()%'})

# Ex Dividend Date
for td in soup.findAll('td', attrs={'data-test': 'EX_DIVIDEND_DATE-value'}):
    for span in td.findAll('span', recursive=False):
        details['Ex Dividend Rate'] = span.text.strip()

# One Year Target Price
for td in soup.findAll('td', attrs={'data-test': 'ONE_YEAR_TARGET_PRICE-value'}):
    for span in td.findAll('span', recursive=False):
        details['One Year Target Price'] = span.text.strip()

# Other Details
profile['Other Details'] = details
profile


{'Present Value': '228.32',
 'Other Details': {'Previous Close': '226.67',
  'Open': '228.50',
  'Bid': '228.24 x 1100',
  'Ask': '228.29 x 900',
  'Day Volume': '1,155,314',
  'Average 3M Volume': '3,752,587',
  'Market Capitalization': '250.039B',
  'Beta 3Y': '1.14',
  'PE Ratio': '22.78',
  'EPS Ratio': '10.02',
  'Earnings Date': '19 Nov 2019',
  'Dividend': '5.44',
  'Dividend Yield': '2.41',
  'Ex Dividend Rate': '2019-09-04',
  'One Year Target Price': '230.85'}}

## <a name="func"></a> [Fundamental Functions](#TOC)
---

Using this experimental code above we can develop functions with better error handling and minimalism.


In [45]:
# ------------------------- FORM QUERY ------------------------- #
# Takes a stock ticker as input and returns the full HTTP request
#   for the yahoo finance query. This is an intermediate function
#   to streamline other processes.
#

def FormQuery(ticker):
    return "https://in.finance.yahoo.com/quote/{}?ltr=1".format(ticker)


# ------------------------- READ HTML ------------------------- #
# Uses a query and returns the prettified HTML for reading.
#

def ReadHTML(query):
    response = requests.get(query)
    soup = BeautifulSoup(response.text, 'lxml')
    print(soup.prettify())
    
    
# ------------------------- PARSE HTML ------------------------- #
# Uses an HTML tree and searches for specific tags which house the
#   stock information. This function utilizes try-catch methods
#   for error handling of each data element. Returns empty for the
#   missing values. The final return is a python dictionary.
#

def ParseHTML(query):
    
    # For ignoring SSL certificate errors
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    # Making the website believe that you are accessing it using a Mozilla browser
    req = Request(query, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    # Creating a BeautifulSoup object of the HTML page for easy extraction of data.
    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    profile = {}
    trading = {}
    fundamentals = {}
    
    # TRADING
    
    # Previous Close
    for td in soup.findAll('td', attrs={'data-test': 'PREV_CLOSE-value'}):
        for span in td.findAll('span', recursive=False):
            trading['Previous Close'] = span.text.strip()
    
    # Open Value
    for td in soup.findAll('td', attrs={'data-test': 'OPEN-value'}):
        for span in td.findAll('span', recursive=False):
            trading['Open'] = span.text.strip()

    # Present Value
    for span in soup.findAll('span', attrs={'class': 'Trsdu(0.3s) Trsdu(0.3s) Fw(b) Fz(36px) Mb(-4px) D(b)'}):
        trading['Present Value'] = span.text.strip()
            
    # Bid
    for td in soup.findAll('td', attrs={'data-test': 'BID-value'}):
        for span in td.findAll('span', recursive=False):
            trading['Bid'] = span.text.strip()

    # Ask
    for td in soup.findAll('td', attrs={'data-test': 'ASK-value'}):
        for span in td.findAll('span', recursive=False):
            trading['Ask'] = span.text.strip()

    # Present Growth
    for div in soup.findAll('div', attrs={'class': 'D(ib) Va(t)'}):
        for span in div.findAll('span', recursive=False):
            profile['Present Growth'] = span.text.strip()

    # Day's Range
    for td in soup.findAll('td', attrs={'data-test': 'DAYS_RANGE-value'}):
        for span in td.findAll('span', recursive=False):
            trading['Day Range'] = span.text.strip()

    # Fifty-two Week Range
    for td in soup.findAll('td', attrs={'data-test': 'FIFTY_TWO_WK_RANGE-value'}):
        for span in td.findAll('span', recursive=False):
            trading['Fifty-Two Week Range'] = span.text.strip()

    # Trading Volume
    for td in soup.findAll('td', attrs={'data-test': 'TD_VOLUME-value'}):
        for span in td.findAll('span', recursive=False):
            trading['Day Volume'] = span.text.strip()

    # Average 3M Volume
    for td in soup.findAll('td', attrs={'data-test': 'AVERAGE_VOLUME_3MONTH-value'}):
        for span in td.findAll('span', recursive=False):
            trading['Average 3M Volume'] = span.text.strip()
            
    # FUNDAMENTALS

    # Market Capitalization
    for td in soup.findAll('td', attrs={'data-test': 'MARKET_CAP-value'}):
        for span in td.findAll('span', recursive=False):
            fundamentals['Market Capitalization'] = span.text.strip()

    # Beta 3Y
    for td in soup.findAll('td', attrs={'data-test': 'BETA_3Y-value'}):
        for span in td.findAll('span', recursive=False):
            fundamentals['Beta 3Y'] = span.text.strip()

    # PE Ratio
    for td in soup.findAll('td', attrs={'data-test': 'PE_RATIO-value'}):
        for span in td.findAll('span', recursive=False):
            fundamentals['PE Ratio'] = span.text.strip()

    # EPS Ratio
    for td in soup.findAll('td', attrs={'data-test': 'EPS_RATIO-value'}):
        for span in td.findAll('span', recursive=False):
            fundamentals['EPS Ratio'] = span.text.strip()

    # Earnings Date
    for td in soup.findAll('td', attrs={'data-test': 'EARNINGS_DATE-value'}):
        trading['Earnings Date'] = []
        for span in td.findAll('span', recursive=False):
            fundamentals['Earnings Date'] = span.text.strip()

    # Dividend and Yield
    for td in soup.findAll('td', attrs={'data-test': 'DIVIDEND_AND_YIELD-value'}):
        fundamentals['Dividend'] = td.text.strip().split()[0]
        fundamentals['Dividend Yield'] = td.text.strip().split()[1].translate({ord(i): None for i in '()%'})

    # Ex Dividend Date
    for td in soup.findAll('td', attrs={'data-test': 'EX_DIVIDEND_DATE-value'}):
        for span in td.findAll('span', recursive=False):
            fundamentals['Ex Dividend Rate'] = span.text.strip()

    # One Year Target Price
    for td in soup.findAll('td', attrs={'data-test': 'ONE_YEAR_TARGET_PRICE-value'}):
        for span in td.findAll('span', recursive=False):
            fundamentals['One Year Target Price'] = span.text.strip()

    # Other Details
    profile['Trading'] = trading
    profile['Fundamental'] = fundamentals
    
    # Return full profile
    return profile


# ------------------------- EXPORT JSON ------------------------- #
# Takes the stock profile as a dictionary and exports the contents
#   as a JSON file using the ticker as the file name.
#

def ExportJSON(profile, ticker):
    data_file = data_path / "{}.json".format(ticker)
    with data_file.open("w") as fp:
        json.dump(profile, fp)


# ------------------------- IMPORT JSON ------------------------- #
# Takes a stock ticker and finds the stock profile JSON before
#   returning the contents of the profile as a dictionary.
#

def ImportJSON(ticker):
    data_file = data_path / "{}.json".format(ticker)
    data_str = open(data_file).read()
    return json.loads(data_str)


In [46]:
# ------------------------- TEST FUNCTIONS ------------------------- #

# Set ticker and form query
ticker = "DOV"
query = FormQuery(ticker)

# Call for profile
profile = ParseHTML(query)

# Export profile as JSON
ExportJSON(profile, ticker)

# Import profile from JSON for inspection
ImportJSON(ticker)

{'Trading': {'Previous Close': '91.99',
  'Open': '93.19',
  'Present Value': '93.22',
  'Bid': '93.29 x 1100',
  'Ask': '93.42 x 1100',
  'Day Volume': '230,907',
  'Average 3M Volume': '845,490',
  'Earnings Date': []},
 'Fundamental': {'Market Capitalization': '13.558B',
  'Beta 3Y': '1.73',
  'PE Ratio': '22.81',
  'EPS Ratio': '4.09',
  'Earnings Date': '21 Oct 2019',
  'Dividend': '1.96',
  'Dividend Yield': '2.13',
  'Ex Dividend Rate': '2019-08-29',
  'One Year Target Price': '107.64'}}

## <a name="REST"></a> [RESTful Functions](#TOC)
---

Combining these basic functions together we can form the fundamentals of a REST API: CREATE, UPDATE, GET, DELETE. Some of these functions are redundant. For instance, the CREATE and UPDATE calls are the same as they both prefer to over-write existing information. This is intentional as it ensures that the profile is as up to date as possible but it might not be the best practice for API design as is makes the over-write decision for the user.


In [47]:
# ------------------------- CREATE PROFILE ------------------------- #
# Scrapes the information from finance.yahoo and updates the JSON
#   stock profile or creates the profile if it did not already exist
#   in the database.
#

def CreateProfile(ticker):
    
    # Form finance.yahoo query
    query = FormQuery(ticker)

    # Call for profile
    profile = ParseHTML(query)

    # Export profile as JSON
    ExportJSON(profile, ticker)
    
    # Return query results
    return json.loads(json.dumps(profile))


# ------------------------- UPDATE PROFILE ------------------------- #
# Creating and updating a profile are ultimately the same function as
#   they both overwrite the existing profile or create it if it does
#   not exist. For this reason, update calls just function-forward to
#   the create call.
#

def UpdateProfile(ticker):
    CreateProfile(ticker)
    
    
# ------------------------- GET PROFILE ------------------------- #
# This function finds the profile within the database and loads the
#   function as a JSON. In verbose mode, this function will also
#   return the dictionary version of the JSON file. In normal mode
#   it returns the JSON string itself, assuming an API application.
#

def GetProfile(ticker):
    return ImportJSON(ticker)



In [48]:
ticker = "LOW"
CreateProfile(ticker)

{'Trading': {'Previous Close': '106.50',
  'Open': '107.19',
  'Present Value': '106.40',
  'Bid': '106.40 x 900',
  'Ask': '106.45 x 800',
  'Day Volume': '1,545,339',
  'Average 3M Volume': '4,671,632',
  'Earnings Date': []},
 'Fundamental': {'Market Capitalization': '82.119B',
  'Beta 3Y': '1.42',
  'PE Ratio': '33.64',
  'EPS Ratio': '3.16',
  'Earnings Date': '20 Nov 2019',
  'Dividend': '2.20',
  'Dividend Yield': '2.07',
  'Ex Dividend Rate': '2019-10-22',
  'One Year Target Price': '121.62'}}

*Written by Austin Dial on 10/08/2019.*